# ML Model to API: Complete Beginner's Guide

This notebook will show you how to take your trained ML model and turn it into a backend API that your Exoplanet Scout web app can communicate with.

**What we'll cover:**
1. Installing required libraries
2. Creating a simple Flask API server
3. Testing your ML function
4. Building API endpoints
5. Testing the API locally
6. Deploying to cloud platforms
7. Creating client functions
8. Error handling

**Prerequisites:** You have a trained ML model that can classify exoplanets based on orbital parameters.

## 1. Install Required Libraries

First, let's install all the libraries we need to create and test our API.

In [ ]:
# Install required libraries
# Run this once to install all dependencies
!pip install flask flask-cors requests numpy pandas scikit-learn

# If you're using other ML libraries, install them too:
# !pip install tensorflow  # for TensorFlow models
# !pip install torch       # for PyTorch models
# !pip install joblib      # for saving/loading models

print("✅ All libraries installed successfully!")

## 2. Your ML Function (Start Here)

Let's assume you have a function like this. **Replace this with your actual trained model and prediction function.**

In [ ]:
# REPLACE THIS WITH YOUR ACTUAL ML FUNCTION
# This is just an example - use your trained model instead

import numpy as np
import joblib  # or whatever you use to load your model

# Example: Load your trained model
# model = joblib.load('your_model.pkl')  # Replace with your model path
# or
# import tensorflow as tf
# model = tf.keras.models.load_model('your_model.h5')

def predict_exoplanet(orbital_period, transit_depth, transit_duration, 
                     signal_to_noise_ratio, stellar_radius, stellar_temperature, 
                     stellar_magnitude):
    """
    Your ML function that predicts if something is an exoplanet.
    
    Input parameters:
    - orbital_period: float (days)
    - transit_depth: float (ppm)
    - transit_duration: float (hours)
    - signal_to_noise_ratio: float
    - stellar_radius: float (solar radii)
    - stellar_temperature: float (Kelvin)
    - stellar_magnitude: float
    
    Returns:
    - string: "exoplanet" or "not_exoplanet"
    """
    
    # REPLACE THIS SECTION WITH YOUR ACTUAL MODEL PREDICTION
    # This is just a dummy example for demonstration
    
    # Prepare input features (adjust based on your model's requirements)
    features = np.array([[orbital_period, transit_depth, transit_duration, 
                         signal_to_noise_ratio, stellar_radius, 
                         stellar_temperature, stellar_magnitude]])
    
    # Example prediction logic (replace with your actual model)
    # prediction = model.predict(features)
    # prediction_proba = model.predict_proba(features)
    
    # Dummy logic for demonstration (REPLACE THIS!)
    if (orbital_period > 0.5 and orbital_period < 500 and 
        transit_depth > 100 and signal_to_noise_ratio > 5):
        return "exoplanet"
    else:
        return "not_exoplanet"

# Test your function
test_result = predict_exoplanet(
    orbital_period=112.3,
    transit_depth=850.2,
    transit_duration=6.25,
    signal_to_noise_ratio=15.8,
    stellar_radius=1.2,
    stellar_temperature=5770,
    stellar_magnitude=13.2
)

print(f"Test prediction: {test_result}")
print("✅ Your ML function is working!")

## 3. Create a Simple Flask API Server

Now let's wrap your ML function in a web API using Flask. This creates a server that can receive HTTP requests.

In [ ]:
# Create a file called 'app.py' with this content
# You can run this cell to create the file automatically

api_code = '''
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
from datetime import datetime
import time

# Import your ML function here
# from your_model_file import predict_exoplanet

app = Flask(__name__)
CORS(app)  # This allows your web app to communicate with the API

# PASTE YOUR predict_exoplanet FUNCTION HERE
def predict_exoplanet(orbital_period, transit_depth, transit_duration, 
                     signal_to_noise_ratio, stellar_radius, stellar_temperature, 
                     stellar_magnitude):
    """Replace this with your actual ML function"""
    # Dummy logic - replace with your model
    if (orbital_period > 0.5 and orbital_period < 500 and 
        transit_depth > 100 and signal_to_noise_ratio > 5):
        return "exoplanet"
    else:
        return "not_exoplanet"

@app.route('/health', methods=['GET'])
def health_check():
    """Simple health check endpoint"""
    return jsonify({"status": "healthy", "timestamp": datetime.now().isoformat()})

@app.route('/api/analyze', methods=['POST'])
def analyze_exoplanets():
    """Main API endpoint that your web app will call"""
    start_time = time.time()
    
    try:
        # Get the JSON data from the request
        data = request.json
        
        # Extract the exoplanet data
        exoplanet_data = data['data']
        
        # Process each exoplanet
        predictions = []
        insights = []
        
        for planet in exoplanet_data:
            # Extract features
            prediction = predict_exoplanet(
                orbital_period=planet['orbital_period'],
                transit_depth=planet['transit_depth'],
                transit_duration=planet['transit_duration'],
                signal_to_noise_ratio=planet['signal_to_noise_ratio'],
                stellar_radius=planet['stellar_radius'],
                stellar_temperature=planet['stellar_temperature'],
                stellar_magnitude=planet['stellar_magnitude']
            )
            
            predictions.append({
                "planetId": planet['name'],
                "prediction": prediction,
                "confidence": 0.85,  # Add real confidence if your model provides it
                "explanation": f"Classification based on orbital and stellar parameters"
            })
        
        # Generate insights
        total_exoplanets = sum(1 for p in predictions if p['prediction'] == 'exoplanet')
        insights.append(f"Found {total_exoplanets} potential exoplanets out of {len(predictions)} candidates")
        
        if total_exoplanets > 0:
            insights.append("Several candidates show strong exoplanet signatures")
        
        # Calculate processing time
        processing_time = int((time.time() - start_time) * 1000)
        
        # Return the results in the format expected by your web app
        response = {
            "success": True,
            "predictions": predictions,
            "insights": insights,
            "recommendations": ["Consider follow-up observations for confirmed exoplanets"],
            "model_version": "1.0.0",
            "timestamp": datetime.now().isoformat(),
            "processing_time": processing_time
        }
        
        return jsonify(response)
        
    except Exception as e:
        # Return error response
        return jsonify({
            "success": False,
            "error": str(e),
            "timestamp": datetime.now().isoformat()
        }), 500

if __name__ == '__main__':
    print("🚀 Starting Exoplanet ML API server...")
    print("📍 API will be available at: http://localhost:8000")
    print("🔍 Health check: http://localhost:8000/health")
    print("🤖 Analyze endpoint: http://localhost:8000/api/analyze")
    app.run(host='0.0.0.0', port=8000, debug=True)
'''

# Save the API code to a file
with open('app.py', 'w') as f:
    f.write(api_code)

print("✅ Created app.py file!")
print("📝 Next: Replace the dummy predict_exoplanet function with your actual ML function")

## 4. Start Your API Server

Now let's start the server. You have two options:

In [ ]:
# Option 1: Run from terminal/command prompt
# Open a new terminal and run: python app.py

# Option 2: Run from Jupyter (not recommended for production)
# Uncomment the lines below to run the server in the notebook
# WARNING: This will block the notebook until you stop the server

import subprocess
import sys

print("🔧 To start your API server:")
print("1. Open a new terminal/command prompt")
print("2. Navigate to this folder")
print("3. Run: python app.py")
print("")
print("OR run this command:")

# Show the command to run
command = f'{sys.executable} app.py'
print(f"Command: {command}")

# Uncomment these lines if you want to start the server from here
# print("Starting server in 3 seconds...")
# import time
# time.sleep(3)
# subprocess.run([sys.executable, 'app.py'])

## 5. Test Your API

Once your server is running, let's test it to make sure it works correctly.

In [ ]:
import requests
import json

# Test data - same format as your web app will send
test_data = {
    "data": [
        {
            "name": "Test Exoplanet 1",
            "orbital_period": 112.3,
            "transit_depth": 850.2,
            "transit_duration": 6.25,
            "signal_to_noise_ratio": 15.8,
            "stellar_radius": 1.2,
            "stellar_temperature": 5770,
            "stellar_magnitude": 13.2
        },
        {
            "name": "Test Candidate 2",
            "orbital_period": 2.5,
            "transit_depth": 50.0,
            "transit_duration": 1.2,
            "signal_to_noise_ratio": 3.2,
            "stellar_radius": 0.8,
            "stellar_temperature": 4200,
            "stellar_magnitude": 15.8
        }
    ],
    "analysis_type": "classification",
    "parameters": {
        "confidence_threshold": 0.8
    }
}

# Test 1: Health check
try:
    response = requests.get('http://localhost:8000/health', timeout=5)
    if response.status_code == 200:
        print("✅ Health check passed!")
        print(f"Response: {response.json()}")
    else:
        print(f"❌ Health check failed: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"❌ Cannot connect to server: {e}")
    print("Make sure your API server is running (python app.py)")

print("\n" + "="*50 + "\n")

# Test 2: Analysis endpoint
try:
    response = requests.post(
        'http://localhost:8000/api/analyze',
        json=test_data,
        headers={'Content-Type': 'application/json'},
        timeout=30
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ Analysis endpoint working!")
        print(f"Success: {result['success']}")
        print(f"Predictions: {len(result['predictions'])}")
        
        for pred in result['predictions']:
            print(f"  - {pred['planetId']}: {pred['prediction']} (confidence: {pred['confidence']})")
        
        print(f"Insights: {result['insights']}")
        print(f"Processing time: {result['processing_time']}ms")
    else:
        print(f"❌ Analysis failed: {response.status_code}")
        print(f"Response: {response.text}")
        
except requests.exceptions.RequestException as e:
    print(f"❌ Cannot connect to analysis endpoint: {e}")

print("\n🎉 If both tests passed, your API is ready to use with your web app!")

## 6. Configure Your Web App

Now you can configure your Exoplanet Scout web app to use your API!

In [ ]:
# Instructions for connecting your web app to the API

print("🔧 To connect your Exoplanet Scout web app to your API:")
print("")
print("1. Make sure your API server is running (python app.py)")
print("2. Open your Exoplanet Scout web app")
print("3. Go to the 'AI Analysis' tab")
print("4. Configure the API endpoint:")
print("   - Endpoint: http://localhost:8000/api/analyze")
print("   - API Key: (leave empty for local server)")
print("   - Model Name: Exoplanet Classifier")
print("   - Timeout: 30000")
print("5. Click 'Test Connection'")
print("6. If successful, you can now run AI analysis on your data!")
print("")
print("📝 Local API URL: http://localhost:8000")
print("🔍 Health check: http://localhost:8000/health")
print("🤖 Analysis endpoint: http://localhost:8000/api/analyze")

## 7. Deploy to Cloud (Optional)

For production use, you'll want to deploy your API to a cloud platform so it's accessible from anywhere.

In [ ]:
# Create deployment files for different cloud platforms

# 1. Create requirements.txt for dependencies
requirements_txt = """flask==2.3.3
flask-cors==4.0.0
requests==2.31.0
numpy==1.24.3
pandas==2.0.3
scikit-learn==1.3.0
gunicorn==21.2.0
"""

with open('requirements.txt', 'w') as f:
    f.write(requirements_txt)

print("✅ Created requirements.txt")

# 2. Create Procfile for Heroku
procfile = """web: gunicorn app:app"""

with open('Procfile', 'w') as f:
    f.write(procfile)

print("✅ Created Procfile for Heroku")

# 3. Instructions for different platforms
print("\n🚀 Cloud Deployment Options:")
print("")
print("OPTION A - Heroku (Free tier available):")
print("1. Create account at heroku.com")
print("2. Install Heroku CLI")
print("3. Run: heroku create your-exoplanet-api")
print("4. Run: git init && git add . && git commit -m 'Initial commit'")
print("5. Run: heroku git:remote -a your-exoplanet-api")
print("6. Run: git push heroku main")
print("Your API will be at: https://your-exoplanet-api.herokuapp.com")
print("")
print("OPTION B - Railway (Easy deployment):")
print("1. Go to railway.app")
print("2. Connect your GitHub repository")
print("3. Deploy automatically")
print("")
print("OPTION C - Google Cloud Run:")
print("1. Create Dockerfile (see next cell)")
print("2. Deploy using Google Cloud Console")
print("")
print("OPTION D - AWS Lambda (Serverless):")
print("1. Use Zappa framework")
print("2. pip install zappa")
print("3. Configure and deploy")

## 8. Error Handling and Troubleshooting

Common issues and how to fix them:

In [ ]:
# Common troubleshooting guide

print("🔧 TROUBLESHOOTING GUIDE")
print("="*50)
print("")

print("❌ Problem: 'Connection refused' or 'Cannot connect'")
print("✅ Solution: Make sure your API server is running")
print("   Run: python app.py")
print("   Check that you see: 'Running on http://127.0.0.1:8000'")
print("")

print("❌ Problem: 'Port already in use'")
print("✅ Solution: Either:")
print("   1. Stop the existing process using that port")
print("   2. Change the port in app.py: app.run(port=8001)")
print("")

print("❌ Problem: 'CORS error' in web browser")
print("✅ Solution: Make sure flask-cors is installed and enabled")
print("   The line 'CORS(app)' should be in your app.py")
print("")

print("❌ Problem: 'Module not found' error")
print("✅ Solution: Install missing dependencies:")
print("   pip install flask flask-cors requests numpy")
print("")

print("❌ Problem: Model prediction errors")
print("✅ Solution: Check your predict_exoplanet function:")
print("   1. Make sure all required model files are in the right location")
print("   2. Test the function separately before wrapping in API")
print("   3. Check input data types and ranges")
print("")

print("❌ Problem: 'KeyError' when processing requests")
print("✅ Solution: Check the data format matches what your function expects")
print("   Print the incoming data to debug: print(request.json)")
print("")

print("🔍 DEBUGGING TIPS:")
print("1. Check the terminal where your API is running for error messages")
print("2. Add print statements to see what data is being received")
print("3. Test your ML function separately before wrapping in API")
print("4. Use the browser developer tools to see network requests")
print("5. Start simple and add complexity gradually")

# Create a simple test script
test_script = '''
# test_api.py - Simple API test script
import requests

def test_api():
    try:
        # Test health check
        response = requests.get('http://localhost:8000/health')
        print(f"Health check: {response.status_code}")
        
        # Test analysis
        test_data = {
            "data": [{
                "name": "Test",
                "orbital_period": 100,
                "transit_depth": 500,
                "transit_duration": 5,
                "signal_to_noise_ratio": 10,
                "stellar_radius": 1.0,
                "stellar_temperature": 5000,
                "stellar_magnitude": 12
            }],
            "analysis_type": "classification"
        }
        
        response = requests.post('http://localhost:8000/api/analyze', json=test_data)
        print(f"Analysis: {response.status_code}")
        if response.status_code == 200:
            print("Success:", response.json())
        else:
            print("Error:", response.text)
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    test_api()
'''

with open('test_api.py', 'w') as f:
    f.write(test_script)

print("\n✅ Created test_api.py - run this to test your API")
print("Usage: python test_api.py")

## 🎉 Summary

You now have everything you need to connect your ML model to your web app!

In [ ]:
# Final checklist and summary

print("🎉 CONGRATULATIONS! You've learned how to:")
print("="*50)
print("✅ Wrap your ML function in a Flask API")
print("✅ Handle HTTP requests and responses")  
print("✅ Test your API locally")
print("✅ Connect your web app to the API")
print("✅ Deploy to cloud platforms")
print("✅ Debug common issues")
print("")

print("📋 QUICK START CHECKLIST:")
print("1. Replace the dummy predict_exoplanet function with your real model")
print("2. Run: python app.py")
print("3. Test with: python test_api.py")
print("4. Configure your web app to use: http://localhost:8000/api/analyze")
print("5. Start analyzing exoplanets!")
print("")

print("🔗 FILES CREATED:")
print("- app.py (Your Flask API server)")
print("- test_api.py (Test script)")
print("- requirements.txt (Dependencies)")
print("- Procfile (For Heroku deployment)")
print("")

print("🌐 LOCAL ENDPOINTS:")
print("- Health check: http://localhost:8000/health")
print("- API endpoint: http://localhost:8000/api/analyze")
print("")

print("📞 NEED HELP?")
print("- Check the troubleshooting section above")
print("- Make sure all dependencies are installed")
print("- Test your ML function separately first")
print("- Check terminal output for error messages")
print("")

print("🚀 Ready to discover exoplanets with AI! 🪐✨")